### Get Committee Names and Codes from Clerk of the House
Works for current Congress (can use archive for prior)

In [9]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

# Send a GET request
response = requests.get('https://clerk.house.gov/committees')

# Parse the HTML content
soup = BeautifulSoup(response.text, 'html.parser')

# Find all the anchor tags
anchor_tags = soup.find_all('a')

# Initialize two empty lists to store the committee names and codes
committee_names = []
codes = []

# For each anchor tag
for tag in anchor_tags:
    # Extract the href attribute
    href = tag.get('href')
    
    # Check if the href attribute contains the path for committees or subcommittees
    if href and ('/committees/' in href or '/subcommittees/' in href):
        # Extract the 4 character code
        code = re.search(r'/([A-Z0-9]{4})$', href)
        
        # If a code was found
        if code:
            # Add the committee name and code to the respective lists
            if '00' in href: #full committee
                committee_names.append(tag.text.strip())
            else:  # '/subcommittees/' in href
                previous_href = tag.find_previous('a', href=re.compile(r'/[A-Z0-9]{2}00$'))
                if previous_href:
                    committee_names.append('Subcommittee on ' + tag.text.strip() + ' (Committee on ' + previous_href.text.strip() + ')')
            codes.append(code.group(1))

# Create a DataFrame
df = pd.DataFrame({
    'Committee': committee_names,
    'Code': codes
})

#remove quotation marks from the df
df['Committee'] = df['Committee'].str.replace('"', '')

# display the DataFrame
display(df)

df.to_csv('replacement118.csv', index=False)

                                             Committee  Code
0                                          Agriculture  AG00
1                                       Appropriations  AP00
2                                       Armed Services  AS00
3                                               Budget  BU00
4                          Education and the Workforce  ED00
..                                                 ...   ...
156  Subcommittee on Defense Intelligence and Overh...  IG04
157  Subcommittee on National Intelligence Enterpri...  IG06
158  Subcommittee on National Security Agency and C...  IG02
159  Subcommittee on Oversight and Investigations (...  IG09
160  Select Committee on the  Strategic Competition...  ZS00

[161 rows x 2 columns]
